<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Semantic Functions 

Semantic Kernel Functions are the core building blocks of functionality for intelligent AI orchestration. Semantic Kernel functions usually have a single responsibility to perform. For example, a Sementic Kernel function can: send an e-mail, call an API, recommend a reasoning framework for a high-stakes decision etc. 

What makes Semantic Kernel functions so special? Can't one just produce similar outcomes by using a prompt in an LLM or just writing a software program? In order to answer that, one has to look at what makes GenAI so innovative. The GenAI models have a unique ability to process instructions with reasoning and logic. This allows these models to behave almost like a human decision maker. Even with basic prompts, GenAI models perform reasonably well. However, providing GenAI models with additional functions allows them to gain access to business processes, data and basically anything a GenAI model should consider when performing AI orchestration. 

For example, imagine you want to prepare a great Thanksgiving dinner and want to get help from a GenAI cooking application to create a new recipe. You enter what you want todo and it comes up with the most delicious looking feast for Thanksgiving. However, there is a problem it used ingredients and recommended using cooking appliances that you do not own! You could enter all of the ingredients and keep prompting until it narrowed down the what you could realistically make. Wouldn't it be better if the GenAI cooking application had access to: your available ingredients, your time availability, kitchen appliances and even your allergic preferences. Now the GenAI model can craft a Thanksgiving feast understanding the parameters and data without having to be guided. This is exactly what Semantic Kernel functions provide.

Semantic Kernel functions come in two flavors: semantic functions and native functions. Semantic functions are basically prompt instructions that help guide GenAI LLMs around the specific guidelines it should adhere to when building the AI orchestration. Therefore, semantic functions are likely the first place most AI architects/engineers start when composing AI orchestration with Semantic Kernel. Semantic functions can also take in parameters for dynamic creaation of instructions. 

Below is an example of semantic function that can be used for facilitating decision-making. Note the {{$decisionToMake}} input where the parameter can be dynamically passed in to provide additional specificity.

```javascript
[DECISION FRAMEWORKS TO USE]
Price's Law
Pareto Principle
Laplace Rule of Succession
Eisenhower Matrix
Median Rule of Five
Second Order Thinking

[DECISION GUIDANCE]
Try to use the best fitting framework.  
Prefer using quantitative decision frameworks rather qualitative ones.  
Use the code interpreter to validate quantitative reasoning.

Use the decision frameworks above to help the user make the following decision:
{{$decisionToMake}}
```  

The image below illustrates the core value of semantic functions. 

<img width ="700px" src="https://learn.microsoft.com/en-us/semantic-kernel/media/semantic-function-explainer.png">

 
 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha

In [2]:
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

### Step 2 - Create a Semantic Kernel - Semantic Function

Semantic functions are constructs inside Semantic Kernel that are basically just prompt instructions. What makes the special is that they can be exposed as functions, thus it makes re-usability easy. Furthermore, future modules will show how they can faciliate composability into plugins. 

In [3]:
var decisionPromptTemplate = "Provide some decision frameworks that can help improve the quality of decisions.";
var semanticDecisionFunction = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate);

await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunction))
{
   Console.Write(streamChunk);
}

1. Decision Matrix: Create a table with options and criteria to evaluate each option objectively. Assign weights to criteria based on importance and score options accordingly.

2. SWOT Analysis: Assess the strengths, weaknesses, opportunities, and threats related to each decision option to identify the most balanced choice.

3. Cost-Benefit Analysis: List the costs and benefits of each option, quantify them, and compare to determine which option offers the greatest net benefit.

4. Decision Tree: Map out possible decisions and their consequences in a tree-like diagram, helping visualize the potential outcomes and choose the path with the highest expected value.

5. Pareto Analysis: Identify the few factors that will have the greatest impact on the decision and focus on options that optimize these key areas.

6. Delphi Technique: Gather input from a panel of experts through multiple rounds of anonymous feedback and use the consensus to guide the decision-making process.

7. Scenario Pla

### Step 3 - Semantic Function Dynamic Execution

Semantic functions can be dynamically composed using Kernel arguments. This allows the ease of passing in parameters and execution settings into the semantic functions.

In [4]:
// Create the OpenAI prompt execution settings
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
    
var decisionPromptTemplate = "Provide some {{$numberOfFrameworks}} frameworks that can help improve the quality of {{$decisionType}} decisions.";
var semanticDecisionFunctionWithInput = semanticKernel.CreateFunctionFromPrompt(decisionPromptTemplate, openAIPromptExecutionSettings);

// Dynamically set the number of frameworks and decision type
var kernelArguments = new KernelArguments();
kernelArguments.Add("numberOfFrameworks", 3);
kernelArguments.Add("decisionType", "strategic");

await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunctionWithInput, kernelArguments))
{
   Console.Write(streamChunk);
}

1. SWOT Analysis: This framework helps organizations assess their internal strengths and weaknesses, as well as external opportunities and threats. By identifying these factors, decision-makers can develop strategies that leverage strengths, address weaknesses, capitalize on opportunities, and mitigate threats.

2. Balanced Scorecard: This framework helps organizations align their strategic objectives with performance metrics across four perspectives: financial, customer, internal processes, and learning and growth. By monitoring and analyzing these metrics, decision-makers can make informed decisions that drive strategic success and improve overall performance.

3. Scenario Planning: This framework involves creating multiple plausible future scenarios based on various external factors and trends. By considering different potential outcomes, decision-makers can develop flexible strategies that are robust under various conditions, thereby improving the quality of strategic decisions and

In [5]:
// Dynamically set the number of frameworks and decision type
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
kernelArguments.Add("numberOfFrameworks", 5);
kernelArguments.Add("decisionType", "requring fast action");

await foreach (var streamChunk in semanticKernel.InvokeStreamingAsync(semanticDecisionFunctionWithInput, kernelArguments))
{
   Console.Write(streamChunk);
}

1. OODA Loop (Observe, Orient, Decide, Act): Developed by military strategist John Boyd, the OODA Loop is a decision-making framework that emphasizes rapid observation and orientation to make quick and effective decisions in fast-paced environments.

2. RAPID Framework: RAPID is an acronym for Recommend, Agree, Perform, Input, and Decide. This framework assigns clear roles and responsibilities to team members, ensuring that decisions are made efficiently and effectively by the right people.

3. The Cynefin Framework: Developed by Dave Snowden, the Cynefin Framework helps leaders understand the context of a situation to make appropriate decisions. It categorizes problems into five domains (simple, complicated, complex, chaotic, and disorder) and suggests decision-making approaches for each.

4. The Decision Matrix: Also known as the Pugh Matrix, this framework helps prioritize and evaluate options based on specific criteria. By scoring each option against weighted criteria, decision-mak